In [ ]:
import pandas as pd
import numpy as np
avaliacao = pd.read_csv('dados/tabelas_originais/aval_financ.csv', sep="►", engine='python', encoding='utf-8')
produtor = pd.read_csv('dados/tabelas_originais/classif_prod_crm.csv', sep="►", engine='python', encoding='utf-8')
movimento = pd.read_csv('dados/tabelas_originais/it_mks_loc_producao.csv', sep="►", engine='python', encoding='utf-8')
avaliacao.estrutura.head(10)

### ``` Aplicação de lambda na coluna log_ativo retornando 1 ou 0 caso x igual a True ```

In [ ]:
avaliacao['log_ativo'] = avaliacao.log_ativo.apply(lambda x: int(x == True))

###  ```  Funcao que transforma o Campo True em inteiro 1 passando as colunas a serem tratadas ```

In [ ]:
categoricas = ['Bloqueado', 'log_emite_receita', 'ind-cheque','ind-protesto', 'ind-forum', 'ind-safras-ant', 'ativo_retencao_inss', 'empregador_rural', 'log_crm']

def trataBols(df, cat):
    for item in cat:
        df[item] =  df[item].apply(lambda x: int(x == True))
    return df
    
    
df_tratado = trataBols(df_avaliacao, categoricas)
df_tratado.head().T

### ``` Dropando colunas desnecessarias ```

In [ ]:
df_tratado.drop(['Unnamed: 0', 'cdn_repres', 'ind-referencia'], axis =1 , inplace=True)

### ``` Aplicação de lambda para retornar 1 caso string Ativo ou 0 caso inativo. ```

In [ ]:
df_tratado.situacao = df_tratado.situacao.apply(lambda x: 1 if x == 'Ativo' else 0)

### ``` Aplicação de lambda para transformar todos os campos em maiusculas ```

In [ ]:
df_tratado.estrutura = df_tratado.estrutura.apply(lambda x: x.upper())

### ``` Cria colunas com as variaveis categoricas e as concatena ao Dataframe pelo axis 1 de Colunas ```

In [ ]:
df_tratado = pd.concat([df_tratado, pd.get_dummies(df_tratado.estrutura, prefix='est_')], axis=1)

In [ ]:
df_tratado = pd.concat([df_tratado, pd.get_dummies(df_tratado.tipo_produtor)], axis=1)

### ``` Concatenação simples ```

In [ ]:
df_tratado = pd.concat([df_tratado, avaliacao['nome-abrev']], axis=1)
df_tratado.head().T

### ``` Exporta como csv ```

In [ ]:
df_tratado.to_csv('dados/avaliacao.csv', index=False)
df_tratado.head()

### ``` Converte em String uma coluna ```

In [ ]:
produtor.cod_produtor = produtor.cod_produtor.astype(str)

### ``` Aplica lambda para substituir '.' por ',' na coluna selecionada ```

In [ ]:
produtor.cod_produtor = produtor.cod_produtor.apply(lambda x: x.replace('.', ','))

### ``` Arredonda os valores da coluna ```

In [ ]:
produtor.cod_produtor = produtor.cod_produtor.round()

### ``` Organiza o Dataframe por sort_values ```

In [ ]:
produtor.sort_values(by='cod_produtor')

### ``` Função para substituir valores em colunas do DataFrame ```

In [ ]:
cols = ['vlr_realizado_fat', 'vlr_potencial_rec', 'vlr_potencial_fat', 'vlr_realizado_rec', 'perc_faturamento', 'perc_recebimento', 'area_cult_prop', 'area_cult_tot', 'vlr_realizado_geral_rec', 'vlr_potencial_geral_rec', 'perc_geral']
def removedots(df , cols):
    for col in cols:
        df[col] = df[col].apply(lambda x: x.replace('.', ''))
        df[col] = df[col].apply(lambda x: x.replace(',', '.'))
        df[col] = df[col].astype(float)
    return df
df_tratado = removedots(produtor, cols)

In [ ]:
def altera(dado):
    if dado == '2013/2013':
        return 1
    if dado == '2013/2014':
        return 2
    if dado == '2014/2014':
        return 3
    if dado == '2014/2015':
        return 4
    if dado == '2015/2015':
        return 5
    if dado == '2015/2016':
        return 6
    if dado == '2016/2016':
        return 7
    if dado == '2016/2017':
        return 8
    if dado == '2017/2017':
        return 9
    if dado == '2017/2018':
        return 10
    if dado == '2018/2018':
        return 11
    if dado == '2018/2019':
        return 12
    if dado == '2019/2019':
        return 13
    if dado == '2019/2020':
        return 14
    if dado == '2020/2020':
        return 15
    if dado == '2020/2021':
        return 16
    if dado == '2021/2021':
        return 17

### ``` Aplica lambda em cada item da coluna safra, chama funcao que retorna valor respectivo  ```

In [ ]:
df_tratado['indice_safra'] = df_tratado.safra.apply(lambda x: altera(x))

### ``` Uso de LOC para filtro ```

In [ ]:
df_tratado.loc[(df_tratado.safra != '2019/2020') & (df_tratado.safra != '2020/2020') & (df_tratado.safra != '2020/2021')]

### ``` Funcao que retorna o coeficiente angular da reta que melhor se ajusta a curva ```

In [ ]:
from sklearn import preprocessing

ids = list(df_tratado['cod_produtor'].unique())
df_tratado['ind_rec'] = 0

df_tratado.head().T

def best_fit(df):
    
    try:
        df_F = df.loc[(df.safra != '2019/2020') & (df.safra != '2020/2020') & (df.safra != '2020/2021')]
        df_F['vlr_rec_normalizado'] = preprocessing.scale(df_F['vlr_realizado_rec'])

        Y = df_F['vlr_rec_normalizado']
        X = df_F['indice_safra']

        xbar = sum(X)/len(X)
        ybar = sum(Y)/len(Y)
        n = len(X) # or len(Y)

        numer = sum([xi*yi for xi,yi in zip(X, Y)]) - n * xbar * ybar
        denum = sum([xi**2 for xi in X]) - n * xbar**2

        b = numer / denum
        a = ybar - b * xbar
    except:
        b = 0
        #a = ybar - b * xbar

    #print('best fit line:\ny = {:.2f} + {:.2f}x'.format(a, b))

    return b

for id_ in ids:
    df_temp = df_tratado.loc[df_tratado.cod_produtor == id_]
    b = best_fit(df_temp)
    df_tratado.loc[df_tratado.cod_produtor==id_, 'ind_rec'] = b
    
df_tratado.head().T

In [ ]:
df_tratado.loc[df_tratado.cod_produtor==2]

In [ ]:
df_tratado['ind_fat'] = 0 

def best_fit(df):
    
    try:
        df_F = df.loc[(df.safra != '2019/2020') & (df.safra != '2020/2020') & (df.safra != '2020/2021')]
        df_F['vlr_fat_normalizado'] = preprocessing.scale(df_F['vlr_realizado_fat'])
        
        Y = df_F['vlr_fat_normalizado']
        X = df_F['indice_safra']

        xbar = sum(X)/len(X)
        ybar = sum(Y)/len(Y)
        n = len(X) # or len(Y)

        numer = sum([xi*yi for xi,yi in zip(X, Y)]) - n * xbar * ybar
        denum = sum([xi**2 for xi in X]) - n * xbar**2


        b = numer / denum
        a = ybar - b * xbar
    except:
        b = 0
        #a = ybar - b * xbar

    #print('best fit line:\ny = {:.2f} + {:.2f}x'.format(a, b))

    return b

for id_ in ids:
    df_temp = df_tratado.loc[df_tratado.cod_produtor == id_]
    b = best_fit(df_temp)
    df_tratado.loc[df_tratado.cod_produtor==id_, 'ind_fat'] = b
    
df_tratado.head().T

In [ ]:
df_tratado.to_csv('dados/produtor_crm.csv')

### ``` Conta nulos ```

In [ ]:
df_tratado.isna().sum()